# test

## include scripts

### include notebook core

In [ ]:
. ../../../../scripts/nbs_header.ps1

### Include core functions script

In [ ]:
. ../../../../scripts/core.ps1

### Include spiral library

In [ ]:
. ../../../../lib/spiral/lib.ps1

## execute project commands

### run notebook with retries using spiral supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "../../../../workspace/target/release/spiral_builder$(_exe) dib --path test.dib --retries 3" } | Invoke-Block

00:00:01   debug #1 run_with_timeout_async / timeout: 500
00:00:01   debug #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "../../../../workspace/target/release/spiral_builder.exe dib --path test.dib --retries 3",
        [||], None, None, true, None)
00:00:01 verbose #2 > 00:00:00   debug #1 spiral_builder.main / args: MutCell(["dib", "--path", "test.dib", "--retries", "3"])
00:00:01 verbose #3 > 00:00:00   debug #2 runtime.execute_with_options / file_name: dotnet / arguments: ["repl", "--exit-after-run", "--run", "c:/home/git/polyglot/apps/spiral/temp/test/test.dib", "--output-path", "c:/home/git/polyglot/apps/spiral/temp/test/test.dib.ipynb"] / options: (None, "dotnet repl --exit-after-run --run "c:/home/git/polyglot/apps/spiral/temp/test/test.dib" --output-path "c:/home/git/polyglot/apps/spiral/temp/test/test.dib.ipynb"", Array(MutCell([("AUTOMATION", "True")])), None, None, false, None)
00:00:04 verbose #4 > >
00:00:04 verbose #5 >

### parse the .dib file into .spi format with dibparser

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00   debug #1 writeDibCode / output: Spi / path: test.dib
00:00:00   debug #2 parseDibCode / output: Spi / file: test.dib


### build .fsx file from .spi using supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00   debug #1 run_with_timeout_async / timeout: 500
00:00:01   debug #2 run_with_timeout_async / timeout: 500
00:00:01   debug #1 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01   debug #2 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01 verbose #3 Supervisor.sendJson / port: 13805 / json: {"FileOpen":{"spiText":"/// # test (Polyglot)\nnominal i = ()\nnominal e = ()\nnominal s = ()\nnomin...0022\u003C/test\u003E\u0022\n","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:01 verbose #4 Supervisor.sendJson / port: 13805 / json: {"BuildFile":{"backend":"Fsharp","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:02   debug #5 buildFile / takeWhileInclusive / path: test.spi / fsxContent: let rec closure0 () () : int32 =
    let v0 : (string -> unit) = System.Console.WriteLine
    let v1 : string = "test"


## compile and format the project

### compile project with fable targeting optimized rust

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.17.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @davidpodhola
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 294ms

Started Fable compilation...

Fable compilation finished in 1651ms

.\test.fsx(7,0): (7,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


### fix formatting issues in the .rs file using regex and set-content

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
    | FixRust `
| Set-Content test.rs

### format the rust code using cargo fmt

In [ ]:
cargo fmt --

## build and test the project

### build the project in release mode using nightly rust compiler

In [ ]:
cargo +nightly build --release

   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> C:\home\git\polyglot\apps\spiral\temp\test\./main.rs:41:8
   |
41 | struct Cart {
   |        ^^^^
   |
   = note: `#[warn(dead_code)]` on by default

  --> C:\home\git\polyglot\apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&mut self, item: &Item) {
   |        ^^^^^^^^^^^

   --> C:\home\git\polyglot\apps\spiral\temp\test\./main.rs:124:4
    |
124 | fn parse_comment(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^^

   --> C:\home\git\polyglot\apps\spiral\temp\test\./main.rs:130:4
    |
130 | fn parse_string(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^

### run release tests with output enabled

In [ ]:
{ cargo +nightly test --release -- --show-output } | Invoke-Block

   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished `release` profile [optimized] target(s) in 26.84s
     Running unittests main.rs (C:\home\git\polyglot\workspace\target\release\deps\spiral_temp_test-19bf3cc4c88f036d.exe)

running 3 tests
test test_parse_number ... ok
test prop_parse_format_idempotent ... ok
test adding_and_then_removing_an_item_from_the_cart_leaves_the_cart_unchanged ... ok

successes:

---- prop_parse_format_idempotent stdout ----
input=Identifier("Mt8YTpECA838Ly")
input=StringLiteral("q6OCb1Vf=Y4<??=qq@<jF*G-V;`q")
input=Identifier("SR")
input=StringLiteral("Vq=/rb'/3:UKc`jX/?`")
input=Integer(-8304438944488586498)
input=Operator("(")
input=Identifier("d5hQDDk9v96zk5ie8Lf7Y8fysOfVh")
input=StringLiteral("R*$72-8 :%k]P=d3%.f1:e.@")
input=Identifier("bV8Q")
input=Comment("9={JsS]bk<-$`b<t$")
input=Identifier("Pc")
i

### execute the binary in release mode

In [ ]:
{ . $ScriptDir/../../../../workspace/target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
